In [59]:
## import SQLite
import sqlite3
## import pandas to for pandas table
import pandas as pd


In [60]:
## helper function to create SQLite table from the csv document

def pd_to_table(dataframe: pd.DataFrame,
                table_name: str,
                database_name: str) -> None:
    ## local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    ## merge the columns from the dataframe by ","
    cols = dataframe.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    ## connect to the Database file
    connection_db = sqlite3.connect(database_name)
    cur = connection_db.cursor()
    logging.info(f'Database - {database_name} created')

    ## create table
    sql_command = f'''CREATE TABLE {table_name} ({cols_string});'''
    cur.execute(sql_command)
    logging.info(f'SQL Table {table_name} created')

    ## uplode the dataframe to the table

    rows_to_upload = dataframe.to_dict(orient='split')['data']
    sql_command = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""
    cur.executemany(sql_command, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')

    #Commit the changes and close the connection
    connection_db.commit()
    connection_db.close()


def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:

    #Connect to the SQL DB
    connection_db = sqlite3.connect(db_name)

    #Execute the SQL query
    cursor = connection_db.execute(sql_query_string)

    #Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    #Close the connection
    connection_db.close()

    #Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

In [61]:
PL_df = pd.read_csv('Packing_List.csv')

PL_df = PL_df.dropna(how='all')

Lulu_df = pd.read_csv('lululemon.csv')

Lulu_df = Lulu_df.dropna(how='all')

In [ ]:
## select desired columns from the dataframe

lulu_list = Lulu_df[["UniqeID", "Style-Option", "Style Description", "Size", "Shipped Amount"]]

lulu_list.rename(columns = {'Style Description':'Style_Description',
                            "Shipped Amount":"Shipped_Amount",
                            "UniqeID": "UniqueID",
                            "Style-Option":"Style_Option"}, inplace = True)

##check lulu list
lulu_list.head()

In [ ]:
## select desired columns from the dataframe
pack_list = PL_df[["Unique Row ID", "Item Description", "Model No.", "Serial No. / Other item info", "Quantity"]]


pack_list.rename(columns = {'Unique Row ID':'UniqueRowID',
                            "Item Description":"ItemDescription",
                            "Model No.": "ModelNo",
                            "Serial No. / Other item info": "SerialNo",}, inplace = True)

pack_list

In [ ]:
##drop table if anything happenes, then recreate the table
conn = sqlite3.connect("default.db")
cur = conn.cursor()
## change the table name to drop if we already created this table in the database
cur.execute("DROP TABLE AMOUNT_INVENTORY")

In [65]:
pd_to_table(pack_list,
            table_name='INVENTORY',
            database_name='default.db')



In [66]:
pd_to_table(lulu_list,
            table_name='LULULEMON',
            database_name='default.db')

In [ ]:
##write SQL command to execute the commands
sql_query_string = """
    SELECT UniqueRowID, SUM(Quantity) as total_Quantity, ItemDescription, ModelNo, SerialNo
    FROM INVENTORY
    GROUP BY ModelNo, SerialNo
"""

# Step 4: Exectue the SQL query
AMOUNT_INVENTORY = sql_query_to_pd(sql_query_string, db_name='default.db')
AMOUNT_INVENTORY


In [68]:
pd_to_table(AMOUNT_INVENTORY,
            table_name='COUNT_INVENTORY',
            database_name='default.db')


In [69]:
sql_query_string = """
    SELECT *
    FROM COUNT_INVENTORY
    JOIN LULULEMON
    ON SerialNo = Style_Option
    AND ModelNo = Size
"""

In [ ]:
result = sql_query_to_pd(sql_query_string, db_name='default.db')
result

In [71]:
final_table = result[["total_Quantity", "Shipped_Amount","SerialNo","Size","Style_Option", "Style_Description"]]

In [72]:
from google.colab import drive
drive.mount('drive')

final_table.to_csv('/content/drive/My Drive/filename.csv', encoding='utf-8', index=False)